<a href="https://colab.research.google.com/github/asosawelford/MIT-Deep-Learning-6.S191/blob/main/Music_generation_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music Generation with RNNs
This notebook aims to generate a Recurring Neural Network (RNN) for music generation. We will make us of music written in [ABC notation](https://en.wikipedia.org/wiki/ABC_notation) to train a model that generates new music. This notebook as a part of a laboratory excercise, during [MIT 6.S191 Introduction to Deep Learning](https://introtodeeplearning.com). Visit their website for more information on ther great course!

# Dependancies
Make sure to install the dependancies and ensure you are using a GPU (Runtime/Change Runtime Type/GPU)


In [ ]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

!pip install mitdeeplearning
import mitdeeplearning as mdl
import numpy as np
import os
import time
import functools
from IPython import display as ipythondisplay
from tqdm import tqdm
!apt-get install abcmidi timidity > /dev/null 2>&1

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0